https://glama.ai/mcp/servers/@adhikasp/mcp-weather

In [1]:
from IPython.display import display, Javascript

display(Javascript('google.colab.kernel.restart()'))


<IPython.core.display.Javascript object>

In [2]:
import sys
!{sys.executable} -m pip install --upgrade "pyautogen>=0.2.20" "autogen-ext[mcp,openai,azure]" git+https://github.com/adhikasp/mcp-weather.git uv --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 10.1 M

In [3]:
import io, ipykernel.iostream
def fake_fileno(self):
    # Return a dummy valid file descriptor (1 is usually stdout)
    return 1
ipykernel.iostream.OutStream.fileno = fake_fileno


In [4]:


import os
import nest_asyncio
import traceback
from autogen_ext.tools.mcp import mcp_server_tools, StdioServerParams
from autogen import AssistantAgent, UserProxyAgent

# Apply nest_asyncio at the beginning
nest_asyncio.apply()

class CancellationToken:
    """A self-contained replica of AutoGen's internal CancellationToken."""
    def __init__(self):
        self._is_cancelled = False
        self._future = None

    def is_cancelled(self) -> bool:
        return self._is_cancelled

    def cancel(self) -> None:
        self._is_cancelled = True
        if self._future and not self._future.done():
            self._future.cancel()

    def link_future(self, future) -> None:
        self._future = future

#
# STEP 4: THE REST OF YOUR WORKING SCRIPT
from google.colab import userdata
os.environ["ACCUWEATHER_API_KEY"] = userdata.get('ACCUWEATHER_API_KEY')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_BASE_URL"] = userdata.get('AZURE_OPENAI_BASE_URL')



from subprocess import PIPE
from autogen_ext.tools.mcp._config import StdioServerParams


params = StdioServerParams(
    command="uvx",
    args=["--from", "git+https://github.com/adhikasp/mcp-weather.git", "mcp-weather"],
    env={"ACCUWEATHER_API_KEY": os.environ["ACCUWEATHER_API_KEY"]},
    read_timeout_seconds=60,
    stdin=PIPE,
    stdout=PIPE,
    stderr=PIPE,
)



tools = await mcp_server_tools(params)
print("Loaded tools:", [t.name for t in tools])

# --- Tool Definitions and Function Map ---
tool_schemas = [
    {
        "type": "function",
        "function": adapter.schema,
    }
    for adapter in tools
]

def make_adapter_func(adapter):
    """
    Creates a robust, asynchronous wrapper that calls the tool's run_json method
    and ensures a valid CancellationToken is always provided.
    """
    async def wrapper(**kwargs):
        try:
            cancellation_token = kwargs.pop("cancellation_token", None)
            # CRITICAL FIX: If the token is None, create an instance of our OWN class.
            if cancellation_token is None:
                cancellation_token = CancellationToken()
            tool_args = kwargs
            return await adapter.run_json(tool_args, cancellation_token)
        except Exception as e:
            traceback.print_exc()
            return f"Error executing tool '{adapter.name}': {e}"
    return wrapper

function_map = {adapter.name: make_adapter_func(adapter) for adapter in tools}

def safe_is_termination_msg(x):
    """Safely checks if a message is a termination message."""
    if x is None:
        return False
    content = x.get("content", None)
    if content is None:
        return False
    return content.rstrip().endswith("TERMINATE")

# --- Agent Definitions ---
llm_config = {
    "config_list": [
        {
            "model": "gpt-4p1-mini-sandbox",
            "api_type": "azure",
            "api_key": os.environ["AZURE_OPENAI_API_KEY"],
            "base_url": os.environ["AZURE_OPENAI_BASE_URL"],
            "api_version": "2024-05-01-preview",
        }
    ],
    "tools": tool_schemas,
}

assistant = AssistantAgent(
    name="WeatherReporter",
    system_message=(
        "You are a helpful weather reporter. Use the functions available to you to answer the user's request. "
    ),
    llm_config=llm_config,
)
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=15,
    is_termination_msg=safe_is_termination_msg,
)

user_proxy.register_function(function_map=function_map)


result = await user_proxy.a_initiate_chat(
    assistant,
    message="Please give me the current weather and next 12-hour forecast for St. Louis, MO.",
)

print("\n--- FINAL CHAT SUMMARY ---")
print(result.summary)


Loaded tools: ['get_hourly_weather']
UserProxy (to WeatherReporter):

Please give me the current weather and next 12-hour forecast for St. Louis, MO.

--------------------------------------------------------------------------------
WeatherReporter (to UserProxy):

***** Suggested tool call (call_vQbGqddzVcjBjj3yqWQR7ZgN): get_hourly_weather *****
Arguments: 
{"location":"St. Louis, MO"}
***********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_hourly_weather...
Call ID: call_vQbGqddzVcjBjj3yqWQR7ZgN
Input arguments: {'location': 'St. Louis, MO'}
UserProxy (to WeatherReporter):

***** Response from calling tool (call_vQbGqddzVcjBjj3yqWQR7ZgN) *****
[TextContent(type='text', text='{\n  "location": "St. Louis",\n  "location_key": "349084",\n  "country": "United States",\n  "current_conditions": {\n    "temperature": {\n      "value": 25.1,\n      "un